In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os

import numpy as np
import pandas as pd
import atecml.data
from tqdm import tqdm

In [2]:
#训练集为第一步build的纬度提升矩阵，并过滤掉unknown标签
train_df = pd.read_pickle('./01_train.dat')
val_df =  pd.read_pickle('./01_test.dat')

#train_df = atecml.data.load_train()
#val_df = atecml.data.load_test()
train_df.loc[train_df.label == 0, 'Fraud'] = 0
train_df.loc[train_df.label != 0, 'Fraud'] = 1


In [3]:
import joblib
predictors =joblib.load('./woe_feature.dat')
#predictors = [x for x in train_df.columns if x not in atecml.data.NOT_FEATURE_SUM]


target = 'Fraud'
DateFold={}

DateFold[0] = set(atecml.data.filter_date(train_df,start_date='2017-09-05',end_date='2017-09-12').index)
DateFold[1] = set(atecml.data.filter_date(train_df,start_date='2017-09-13',end_date='2017-09-20').index)
DateFold[2] = set(atecml.data.filter_date(train_df,start_date='2017-09-21',end_date='2017-09-28').index)
DateFold[3] = set(atecml.data.filter_date(train_df,start_date='2017-09-29',end_date='2017-10-06').index)
DateFold[4] = set(atecml.data.filter_date(train_df,start_date='2017-10-07',end_date='2017-10-14').index)
DateFold[5] = list(atecml.data.filter_date(train_df,start_date='2017-10-15',end_date='2017-11-24').index)

all_list = set(train_df.index) - set(DateFold[5])
len(all_list),len(DateFold[5])

(634284, 360447)

In [9]:
X = np.array(train_df[predictors])
Y = np.array(train_df[target])

val_index = DateFold[5] 
train_index = list(all_list)

X_train = X[train_index]
y_train = Y[train_index]

X_val = X[val_index]
y_val = Y[val_index]
    
X_test = np.array(val_df[predictors])
print(x_train.shape, y_train.shape, x_test.shape)

(994731, 168) (634284,) (500538, 168)


In [13]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

print('设置参数')
params = {
          'boosting_type': 'gbdt',
          'objective': 'binary',
          'metric': 'binary_logloss',
          }

### 交叉验证(调参)
print('交叉验证')
min_merror = float('Inf')
best_params = {}

# 准确率
print("调参1：提高准确率")
for num_leaves in range(20,300,20):
    for max_depth in range(3,10,2):
        params['num_leaves'] = num_leaves
        params['max_depth'] = max_depth

        cv_results = lgb.cv(
                            params,
                            lgb_train,
                            seed=2018,
                            nfold=5,
                            metrics=['auc'],
                            early_stopping_rounds=10,
                            verbose_eval=True
                            )
            
        mean_merror = pd.Series(cv_results['auc-mean']).min()
        boost_rounds = pd.Series(cv_results['auc-mean']).argmin()
            
        if mean_merror < min_merror:
            min_merror = mean_merror
            best_params['num_leaves'] = num_leaves
            best_params['max_depth'] = max_depth
            
params['num_leaves'] = best_params['num_leaves']
params['max_depth'] = best_params['max_depth']

设置参数
交叉验证
调参1：提高准确率
[1]	cv_agg's auc: 0.780597 + 0.0224243
[2]	cv_agg's auc: 0.89849 + 0.002971
[3]	cv_agg's auc: 0.917358 + 0.00362825
[4]	cv_agg's auc: 0.922178 + 0.00788643
[5]	cv_agg's auc: 0.930729 + 0.00643625
[6]	cv_agg's auc: 0.937408 + 0.0028038
[7]	cv_agg's auc: 0.938361 + 0.00289214
[8]	cv_agg's auc: 0.939351 + 0.00226948
[9]	cv_agg's auc: 0.940343 + 0.00279504
[10]	cv_agg's auc: 0.942422 + 0.00376437
[11]	cv_agg's auc: 0.945006 + 0.00342629
[12]	cv_agg's auc: 0.94762 + 0.00256892
[13]	cv_agg's auc: 0.949398 + 0.00251582
[14]	cv_agg's auc: 0.950335 + 0.00199219
[15]	cv_agg's auc: 0.951277 + 0.00186189
[16]	cv_agg's auc: 0.952062 + 0.00162478
[17]	cv_agg's auc: 0.952763 + 0.00170049
[18]	cv_agg's auc: 0.953487 + 0.00171994
[19]	cv_agg's auc: 0.954162 + 0.00162687
[20]	cv_agg's auc: 0.95474 + 0.00168511
[21]	cv_agg's auc: 0.95527 + 0.00175751
[22]	cv_agg's auc: 0.955679 + 0.00169774
[23]	cv_agg's auc: 0.95604 + 0.001678
[24]	cv_agg's auc: 0.956401 + 0.00172773
[25]	cv_agg's au

[2]	cv_agg's auc: 0.931126 + 0.00409838
[3]	cv_agg's auc: 0.937005 + 0.00469841
[4]	cv_agg's auc: 0.947156 + 0.00274602
[5]	cv_agg's auc: 0.948964 + 0.00313248
[6]	cv_agg's auc: 0.950799 + 0.00349516
[7]	cv_agg's auc: 0.954493 + 0.00238002
[8]	cv_agg's auc: 0.957355 + 0.00254372
[9]	cv_agg's auc: 0.958846 + 0.0027262
[10]	cv_agg's auc: 0.959779 + 0.00290787
[11]	cv_agg's auc: 0.960576 + 0.00273306
[12]	cv_agg's auc: 0.961242 + 0.00254474
[13]	cv_agg's auc: 0.961942 + 0.0025535
[14]	cv_agg's auc: 0.96237 + 0.0025241
[15]	cv_agg's auc: 0.962803 + 0.00249336
[16]	cv_agg's auc: 0.963344 + 0.0023103
[17]	cv_agg's auc: 0.963873 + 0.00229125
[18]	cv_agg's auc: 0.964619 + 0.0024542
[19]	cv_agg's auc: 0.96528 + 0.00225091
[20]	cv_agg's auc: 0.965697 + 0.00197513
[21]	cv_agg's auc: 0.966217 + 0.00177571
[22]	cv_agg's auc: 0.96646 + 0.00175319
[23]	cv_agg's auc: 0.966708 + 0.00169847
[24]	cv_agg's auc: 0.966968 + 0.00165964
[25]	cv_agg's auc: 0.967276 + 0.0016818
[26]	cv_agg's auc: 0.967525 + 0.0

[4]	cv_agg's auc: 0.947545 + 0.00354826
[5]	cv_agg's auc: 0.949196 + 0.00391513
[6]	cv_agg's auc: 0.954252 + 0.00365579
[7]	cv_agg's auc: 0.957345 + 0.00328147
[8]	cv_agg's auc: 0.958487 + 0.00255238
[9]	cv_agg's auc: 0.959462 + 0.00290805
[10]	cv_agg's auc: 0.960258 + 0.00267714
[11]	cv_agg's auc: 0.96094 + 0.00276599
[12]	cv_agg's auc: 0.962557 + 0.00253819
[13]	cv_agg's auc: 0.963446 + 0.00256908
[14]	cv_agg's auc: 0.963799 + 0.00234999
[15]	cv_agg's auc: 0.964483 + 0.00209832
[16]	cv_agg's auc: 0.964876 + 0.00197796
[17]	cv_agg's auc: 0.965504 + 0.00168208
[18]	cv_agg's auc: 0.96592 + 0.00161645
[19]	cv_agg's auc: 0.96646 + 0.00179808
[20]	cv_agg's auc: 0.966772 + 0.00180348
[21]	cv_agg's auc: 0.967233 + 0.0017459
[22]	cv_agg's auc: 0.967739 + 0.00167402
[23]	cv_agg's auc: 0.968117 + 0.00175951
[24]	cv_agg's auc: 0.968433 + 0.00165624
[25]	cv_agg's auc: 0.968768 + 0.00169343
[26]	cv_agg's auc: 0.968992 + 0.00155576
[27]	cv_agg's auc: 0.969175 + 0.00156211
[28]	cv_agg's auc: 0.96934

[6]	cv_agg's auc: 0.954677 + 0.00269065
[7]	cv_agg's auc: 0.957986 + 0.00249965
[8]	cv_agg's auc: 0.9587 + 0.00225168
[9]	cv_agg's auc: 0.959924 + 0.00265265
[10]	cv_agg's auc: 0.960843 + 0.00253709
[11]	cv_agg's auc: 0.962462 + 0.00262223
[12]	cv_agg's auc: 0.963315 + 0.00243809
[13]	cv_agg's auc: 0.963978 + 0.00246094
[14]	cv_agg's auc: 0.964455 + 0.00213509
[15]	cv_agg's auc: 0.964819 + 0.00217678
[16]	cv_agg's auc: 0.965331 + 0.00195933
[17]	cv_agg's auc: 0.966188 + 0.00190034
[18]	cv_agg's auc: 0.966574 + 0.00179752
[19]	cv_agg's auc: 0.967125 + 0.00200668
[20]	cv_agg's auc: 0.967426 + 0.00191248
[21]	cv_agg's auc: 0.967814 + 0.00197886
[22]	cv_agg's auc: 0.968232 + 0.0019584
[23]	cv_agg's auc: 0.968612 + 0.00181067
[24]	cv_agg's auc: 0.969059 + 0.00172036
[25]	cv_agg's auc: 0.969306 + 0.00175923
[26]	cv_agg's auc: 0.969511 + 0.00175966
[27]	cv_agg's auc: 0.969704 + 0.00167997
[28]	cv_agg's auc: 0.969955 + 0.00162942
[29]	cv_agg's auc: 0.970175 + 0.00166543
[30]	cv_agg's auc: 0.97

[8]	cv_agg's auc: 0.950547 + 0.00393642
[9]	cv_agg's auc: 0.952484 + 0.00274284
[10]	cv_agg's auc: 0.954096 + 0.00239577
[11]	cv_agg's auc: 0.954925 + 0.00215539
[12]	cv_agg's auc: 0.956258 + 0.00238267
[13]	cv_agg's auc: 0.957091 + 0.00196117
[14]	cv_agg's auc: 0.95775 + 0.00217401
[15]	cv_agg's auc: 0.958685 + 0.00182998
[16]	cv_agg's auc: 0.959668 + 0.00183967
[17]	cv_agg's auc: 0.960132 + 0.00175455
[18]	cv_agg's auc: 0.960833 + 0.00168243
[19]	cv_agg's auc: 0.961229 + 0.00169395
[20]	cv_agg's auc: 0.961821 + 0.00182325
[21]	cv_agg's auc: 0.962274 + 0.00171953
[22]	cv_agg's auc: 0.962757 + 0.00171113
[23]	cv_agg's auc: 0.963257 + 0.00154164
[24]	cv_agg's auc: 0.963618 + 0.00141008
[25]	cv_agg's auc: 0.963977 + 0.0014253
[26]	cv_agg's auc: 0.96426 + 0.00140667
[27]	cv_agg's auc: 0.964572 + 0.00146001
[28]	cv_agg's auc: 0.964809 + 0.00141687
[29]	cv_agg's auc: 0.964997 + 0.00145987
[30]	cv_agg's auc: 0.965138 + 0.0015343
[31]	cv_agg's auc: 0.965351 + 0.00152372
[32]	cv_agg's auc: 0.9

...

[81]	cv_agg's auc: 0.975307 + 0.00174647
[82]	cv_agg's auc: 0.975308 + 0.00169067
[83]	cv_agg's auc: 0.975345 + 0.00170826
[84]	cv_agg's auc: 0.97536 + 0.00172729
[85]	cv_agg's auc: 0.975391 + 0.00170374
[86]	cv_agg's auc: 0.975411 + 0.00170599
[87]	cv_agg's auc: 0.975446 + 0.00169288
[88]	cv_agg's auc: 0.975441 + 0.00169086
[89]	cv_agg's auc: 0.975488 + 0.00170693
[90]	cv_agg's auc: 0.975515 + 0.00172936
[91]	cv_agg's auc: 0.975526 + 0.001728
[92]	cv_agg's auc: 0.975556 + 0.00172981
[93]	cv_agg's auc: 0.975576 + 0.00175281
[94]	cv_agg's auc: 0.975593 + 0.00176112
[95]	cv_agg's auc: 0.975608 + 0.00179622
[96]	cv_agg's auc: 0.975628 + 0.00180261
[97]	cv_agg's auc: 0.975657 + 0.00178043
[98]	cv_agg's auc: 0.975679 + 0.00178952
[99]	cv_agg's auc: 0.975706 + 0.00176637
[100]	cv_agg's auc: 0.975731 + 0.00179548
[1]	cv_agg's auc: 0.900403 + 0.00269018
[2]	cv_agg's auc: 0.938204 + 0.00278445
[3]	cv_agg's auc: 0.948701 + 0.00617424
[4]	cv_agg's auc: 0.955962 + 0.00507663
[5]	cv_agg's auc: 0.95

[5]	cv_agg's auc: 0.930729 + 0.00643625
[6]	cv_agg's auc: 0.937408 + 0.0028038
[7]	cv_agg's auc: 0.938361 + 0.00289214
[8]	cv_agg's auc: 0.939351 + 0.00226948
[9]	cv_agg's auc: 0.940343 + 0.00279504
[10]	cv_agg's auc: 0.942422 + 0.00376437
[11]	cv_agg's auc: 0.945006 + 0.00342629
[12]	cv_agg's auc: 0.94762 + 0.00256892
[13]	cv_agg's auc: 0.949398 + 0.00251582
[14]	cv_agg's auc: 0.950335 + 0.00199219
[15]	cv_agg's auc: 0.951277 + 0.00186189
[16]	cv_agg's auc: 0.952062 + 0.00162478
[17]	cv_agg's auc: 0.952763 + 0.00170049
[18]	cv_agg's auc: 0.953487 + 0.00171994
[19]	cv_agg's auc: 0.954162 + 0.00162687
[20]	cv_agg's auc: 0.95474 + 0.00168511
[21]	cv_agg's auc: 0.95527 + 0.00175751
[22]	cv_agg's auc: 0.955679 + 0.00169774
[23]	cv_agg's auc: 0.95604 + 0.001678
[24]	cv_agg's auc: 0.956401 + 0.00172773
[25]	cv_agg's auc: 0.956738 + 0.00172683
[26]	cv_agg's auc: 0.957008 + 0.00167781
[27]	cv_agg's auc: 0.957359 + 0.0017577
[28]	cv_agg's auc: 0.957647 + 0.00177683
[29]	cv_agg's auc: 0.957957 +

[7]	cv_agg's auc: 0.956136 + 0.00303704
[8]	cv_agg's auc: 0.958379 + 0.00296719
[9]	cv_agg's auc: 0.960091 + 0.00240692
[10]	cv_agg's auc: 0.960794 + 0.00245461
[11]	cv_agg's auc: 0.961518 + 0.00230889
[12]	cv_agg's auc: 0.962281 + 0.00245873
[13]	cv_agg's auc: 0.96319 + 0.00247438
[14]	cv_agg's auc: 0.963748 + 0.00230711
[15]	cv_agg's auc: 0.964397 + 0.00232417
[16]	cv_agg's auc: 0.964938 + 0.00234677
[17]	cv_agg's auc: 0.965448 + 0.00234677
[18]	cv_agg's auc: 0.966152 + 0.0022455
[19]	cv_agg's auc: 0.966757 + 0.00217778
[20]	cv_agg's auc: 0.967147 + 0.00220999
[21]	cv_agg's auc: 0.967629 + 0.00191102
[22]	cv_agg's auc: 0.967919 + 0.00178814
[23]	cv_agg's auc: 0.968361 + 0.00180745
[24]	cv_agg's auc: 0.968597 + 0.00188664
[25]	cv_agg's auc: 0.968847 + 0.00179294
[26]	cv_agg's auc: 0.969058 + 0.00176901
[27]	cv_agg's auc: 0.969291 + 0.00175089
[28]	cv_agg's auc: 0.969375 + 0.00174241
[29]	cv_agg's auc: 0.969528 + 0.00175756
[30]	cv_agg's auc: 0.96974 + 0.00172835
[31]	cv_agg's auc: 0.9

[9]	cv_agg's auc: 0.966323 + 0.00179856
[10]	cv_agg's auc: 0.966947 + 0.00164937
[11]	cv_agg's auc: 0.967199 + 0.00162861
[12]	cv_agg's auc: 0.967913 + 0.0018694
[13]	cv_agg's auc: 0.968485 + 0.00207675
[14]	cv_agg's auc: 0.968899 + 0.00204744
[15]	cv_agg's auc: 0.969158 + 0.00211481
[16]	cv_agg's auc: 0.969606 + 0.00216354
[17]	cv_agg's auc: 0.970008 + 0.00204362
[18]	cv_agg's auc: 0.970206 + 0.00201004
[19]	cv_agg's auc: 0.970464 + 0.0020014
[20]	cv_agg's auc: 0.970784 + 0.00182018
[21]	cv_agg's auc: 0.971183 + 0.00183918
[22]	cv_agg's auc: 0.971403 + 0.0019181
[23]	cv_agg's auc: 0.971883 + 0.00154199
[24]	cv_agg's auc: 0.972096 + 0.0015461
[25]	cv_agg's auc: 0.972472 + 0.00155587
[26]	cv_agg's auc: 0.972607 + 0.00152302
[27]	cv_agg's auc: 0.972787 + 0.00152915
[28]	cv_agg's auc: 0.973003 + 0.00153193
[29]	cv_agg's auc: 0.973106 + 0.00151978
[30]	cv_agg's auc: 0.973317 + 0.00156754
[31]	cv_agg's auc: 0.973446 + 0.00157357
[32]	cv_agg's auc: 0.973575 + 0.00153204
[33]	cv_agg's auc: 0.

KeyboardInterrupt: 

In [5]:
num_pos = np.sum(train_df[target])  
num_neg = x_train.shape[0]- num_pos
scale_pos_weight =  num_neg/num_pos
print(num_pos,num_neg,scale_pos_weight)

16847.0 977884.0 58.04499317385885


In [ ]:
lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train,free_raw_data=False)

In [ ]:
num_boost_round = 2000
num_early_stop = 100

In [ ]:

dart_classifier = LGBClassifier(boost_type='dart',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
dart_oof_train, dart_oof_test,dart_model_list = dart_classifier.get_oof(x_train, y_train, x_test)
print(dart_oof_train.shape, dart_oof_test.shape)   


gbdt_classifier = LGBClassifier(boost_type='gbdt',boost_round=num_boost_round,early_stop=num_early_stop,pos_weight= scale_pos_weight)
gbdt_oof_train, gbdt_oof_test,gbdt_model_list = gbdt_classifier.get_oof(x_train, y_train, x_test)
print(gbdt_oof_train.shape, gbdt_oof_test.shape)  
 


{'min_child_samples': 20, 'metric': {'auc'}, 'num_leaves': 80, 'subsample': 0.85, 'task': 'train', 'nthread': 40, 'is_unbalance': True, 'learning_rate': 0.05, 'objective': 'binary', 'subsample_freq': 1, 'subsample_for_bin': 2000, 'boosting_type': 'dart', 'verbose': -1, 'use_missing': 'true', 'min_split_gain': 0, 'max_bin': 511, 'colsample_bytree': 0.7, 'max_depth': -1}
0 fold, train 500685, val 360447
train with lgb model
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.967054
[100]	valid_0's auc: 0.97075
[150]	valid_0's auc: 0.973024
[200]	valid_0's auc: 0.974198
[250]	valid_0's auc: 0.975055
[300]	valid_0's auc: 0.975691
[350]	valid_0's auc: 0.976046
[400]	valid_0's auc: 0.976443
[450]	valid_0's auc: 0.97675
[500]	valid_0's auc: 0.976812
[550]	valid_0's auc: 0.976832
[600]	valid_0's auc: 0.977054
[650]	valid_0's auc: 0.977077
[700]	valid_0's auc: 0.977027
[750]	valid_0's auc: 0.976731
Early stopping, best iteration is:
[690]	valid_0's auc: 0.977111

In [ ]:
train_dart_r = gbdt_oof_train
n = int(gbdt_oof_train.shape[0] * 0.8)
y_pred, y_val = train_dart_r[n:], y_train[n:]
_f1,_f2,_f3 = atecml.data.accuracy_validation(y_val,y_pred)


In [ ]:
train_dart_r = dart_oof_train
n = int(dart_oof_train.shape[0] * 0.8)
y_pred, y_val = train_dart_r[n:], y_train[n:]
_f1,_f2,_f3 = atecml.data.accuracy_validation(y_val,y_pred)